In [2]:
import urllib3
import json
import random
import time
import os
import logging

In [11]:
logger = logging.getLogger('crawler_application')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('crawler.log')
ch = logging.StreamHandler()

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [30]:
out_folder = 'regione-toscana'
file_name = 'regione-toscana-result-atti-dirigenti-{:d}-{:d}.json'

In [20]:
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [21]:
http_pool = urllib3.PoolManager()

In [22]:
url_base = "http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start={:d}&num={:d}&filter=0&rc=1&q=inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD"

In [23]:
url_base = "http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start={}&num={}&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DAD+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD"

In [24]:
url_base = "http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start={}&num={}&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DPG+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD"

In [28]:
url_base = "http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start={}&num={}&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DMON+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD"

In [29]:
def download_data(start, num):
    logger.info("start {}, num {}".format(start,num))
    sleep_time = random.randint(2,5)
    logger.debug('sleeping {} seconds'.format(sleep_time))
    time.sleep(random.randint(5,20))
    
    output_name = out_folder + "/" + file_name.format(start, start + num)
    url = url_base.format(start, num)
    logger.info('requesting url {}'.format(url))
    res = http_pool.request('GET', url, retries=1)
    logger.info('got status {} for url {}'.format(res.status, url))
    
    num_res = 0
    
    if res.status is 200:
        json_data = json.loads(res.data.decode('utf-8'))
        if 'RES' in json_data['GSP']:
            num_res = len(json_data['GSP']['RES']['R'])
            with open(output_name, 'w') as f:
                f.write(res.data.decode('utf-8'))
                logger.info('saved result in {}'.format(output_name))
        else:
            logger.error('no results skipped {} \n {}'.format(url, json_data))
    else:
        logger.error('status {} skipped {}'.format(res.status, url))
    
    status = res.status
    res.close()
    return num_res
    

In [31]:
start = 0
num = 100

num_records = 1
counter = 0

while num_records > 0:
    num_records = download_data(start, num)
    counter += num_records
    start += num
    logger.info('extracted {} records'.format(counter))


2018-03-04 22:09:52,960 - crawler_application - INFO - start 0, num 100
2018-03-04 22:09:52,960 - crawler_application - INFO - start 0, num 100
2018-03-04 22:09:52,961 - crawler_application - DEBUG - sleeping 4 seconds
2018-03-04 22:09:52,961 - crawler_application - DEBUG - sleeping 4 seconds
2018-03-04 22:10:07,977 - crawler_application - INFO - requesting url http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=0&num=100&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DMON+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD
2018-03-04 22:10:07,977 - crawler_application - INFO - requesting url http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=0&num=100&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DMON+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD


2018-03-04 22:10:57,362 - crawler_application - INFO - saved result in regione-toscana/regione-toscana-result-atti-dirigenti-300-400.json
2018-03-04 22:10:57,362 - crawler_application - INFO - saved result in regione-toscana/regione-toscana-result-atti-dirigenti-300-400.json
2018-03-04 22:10:57,365 - crawler_application - INFO - extracted 400 records
2018-03-04 22:10:57,365 - crawler_application - INFO - extracted 400 records
2018-03-04 22:10:57,368 - crawler_application - INFO - start 400, num 100
2018-03-04 22:10:57,368 - crawler_application - INFO - start 400, num 100
2018-03-04 22:10:57,371 - crawler_application - DEBUG - sleeping 3 seconds
2018-03-04 22:10:57,371 - crawler_application - DEBUG - sleeping 3 seconds
2018-03-04 22:11:15,390 - crawler_application - INFO - requesting url http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=400&num=100&filter=0&rc=1&q=inmeta%3AID_TIPO_PRA

2018-03-04 22:12:06,410 - crawler_application - INFO - got status 200 for url http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=700&num=100&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DMON+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD
2018-03-04 22:12:06,420 - crawler_application - INFO - saved result in regione-toscana/regione-toscana-result-atti-dirigenti-700-800.json
2018-03-04 22:12:06,420 - crawler_application - INFO - saved result in regione-toscana/regione-toscana-result-atti-dirigenti-700-800.json
2018-03-04 22:12:06,425 - crawler_application - INFO - extracted 800 records
2018-03-04 22:12:06,425 - crawler_application - INFO - extracted 800 records
2018-03-04 22:12:06,428 - crawler_application - INFO - start 800, num 100
2018-03-04 22:12:06,428 - crawler_application - INFO - start 800, num 100
2018-03-04 22:12:06,431 - crawler_application - DEBUG - sle

2018-03-04 22:12:47,801 - crawler_application - ERROR - no results skipped http://www.regione.toscana.it/bancadati/search?site=atti&client=fend_json&output=xml_no_dtd&getfields=*&ulang=it&ie=UTF-8&proxystylesheet=fend_json&start=1000&num=100&filter=0&rc=1&q=inmeta%3AID_TIPO_PRATICA%3DMON+AND+inmeta%3AALLEGATO_DESCRIZIONE%3Dvoid&sort=meta%3ACODICE_PRATICA%3AD 
 {'GSP': {'script_report_load': '/click?&q=inmeta%3AID_TIPO_PRATICA%253DMON%2BAND%2Binmeta%3AALLEGATO_DESCRIZIONE%253Dvoid&ct=load&s=1000&site=atti', 'TM': '0.000801', 'Q': 'inmeta:ID_TIPO_PRATICA=MON AND inmeta:ALLEGATO_DESCRIZIONE=void', 'PARAM': [{'site': {'value': 'atti', 'original_value': 'atti'}, 'client': {'value': 'fend_json', 'original_value': 'fend_json'}, 'output': {'value': 'xml_no_dtd', 'original_value': 'xml_no_dtd'}, 'getfields': {'value': '*', 'original_value': '*'}, 'ulang': {'value': '', 'original_value': ''}, 'ie': {'value': 'UTF-8', 'original_value': 'UTF-8'}, 'proxystylesheet': {'value': 'fend_json', 'original

In [ ]:
#json_data['GSP']['RES']['R'][1]

In [ ]:
http_pool.clear()